<a href="https://colab.research.google.com/github/gwillig/finance/blob/master/on_vista.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import datetime

In [6]:
def clean_name(x):
    """clean the name to convert to something readable
    @:param x(string): e.g.'Bunde­­srep.­­Deuts­­chlan­­d · WKN 110490 · ISIN DE0001104909'

    """

    '#1.Step: Replace the "Soft Hyphen"'
    data = x.replace('\xad', '')
    data = data.replace('\u00ad', '')
    data = data.replace('\N{SOFT HYPHEN}', '')
    '#2.Step: Replace the non-breaking'
    data = data.replace('\xa0', '')
    data = data.replace('\u00a0', '')
    data = data.replace('\N{No-Break Space}', '')

    return data


def split_colum(x):
    """
    split column into name and isin

    :param x:
    :return:
    """
    name = x.split("Anleihe")[0]
    isin = x.split("ISIN")[-1]


    return pd.Series([name, isin])

In [7]:
# Downloading contents of the web page

#1.Step: Download the most search bonds 
url = "https://www.onvista.de/top-werte/top-anleihen"
df_pandas = pd.read_html(url, attrs = {'class': 'table ov-table ov-table--fixed-x-cells outer-spacing--none'},  flavor='bs4')
df = df_pandas[0]

df.keys()
#2.Step: Convert the column "Fälligkeit" from type object to datetime
df['Fälligkeit'] = pd.to_datetime(df['Fälligkeit'], format='%d.%m.%Y')


In [10]:
'#2.Step: Preprocessing'
df["Wert"] = df["Wert"].apply(clean_name)
df[['name', 'isin']] = df["Wert"].apply(split_colum)
#Change order of columns
df = df.reindex(columns=['name',"isin",'Anleihen-TypZins-Typ','Kupon','Nominal','Fälligkeit', 'Rendite'])

In [11]:
# Enrichment
#1. Add to each index the current value
#2. Add to each index the S&Rating
#3. Add to each index the sales value of the last day
datetime_str = '01.01.2024'

datetime_object = datetime.datetime.strptime(datetime_str, '%d.%m.%Y')

df[(df['Fälligkeit']<datetime_object)]

,name,isin,Anleihen-TypZins-Typ,Kupon,Nominal,Fälligkeit,Rendite
1,R-Logitech S.A.M. EO-Anl. 2018(21/23),DE000A19WVN8,AnleiheFest,"8,50 %","1.000,00",2023-03-29,NaN
3,Bundesrep.Deutschland Bundesschatzanw. v.21(23),DE0001104867,AnleiheFest,"0,00 %",001,2023-12-15,"+2,79 %"
10,Bertelsmann SE & Co. KGaA Genußscheine 2001,DE0005229942,GenussscheinFest,"15,00 %",1000,2023-03-19,NaN
14,Bundesrep.Deutschland Bundesobl.Ser.178 v.2018...,DE0001141786,AnleiheFest,NaN,001,2023-10-13,"+2,72 %"
15,Südzucker Intl Finance B.V. EO-FLR Bonds 2005(...,XS0222524372,AnleiheVariabel,"5,30 %","1.000,00",2023-03-19,NaN
17,United States of America DL-Bonds 1993(23),US912810EQ77,AnleiheFest,"6,25 %",10000,2023-08-15,"+4,24 %"
20,Bundesrep.Deutschland Bundesschatzanw. v.21(23),DE0001104859,AnleiheFest,"0,00 %",001,2023-09-15,"+2,76 %"
22,Heckler & Koch GmbH,XS1649057640,AnleiheFest,"6,50 %","1.000,00",2023-04-30,"+6,12 %"
23,Bundesrep.Deutschland Anl.v.2013 (2023),DE0001102325,AnleiheFest,"2,00 %",001,2023-08-15,"+2,40 %"
25,Metalcorp Group S.A. EO-,DE000A19MDV0,AnleiheStufenzins,"8,50 %","1.000,00",2023-10-02,NaN


In [ ]:
##Add rating
a = requests.get("https://disclosure.spglobal.com/ratings/en/regulatory/org-details/sectorCode/CORP/entityId/679156")
a.text

'<!DOCTYPE html><html lang="en"><head>\n  <meta name="google-site-verification" content="YXuDGRNYF3Ihgx93nDneXyKuwH3vzlfWDyL2_OZuJtE">\n  <style ng-transition="serverApp"></style><style ng-transition="serverApp">.text-purple[_ngcontent-sc122]{color:#a868bd}.text-bold[_ngcontent-sc122]{font-family:sans-serif!important}@media screen and (min-width: 75em){.rutimezone[_ngcontent-sc122]{font-weight:bold;text-align:left}}@media only screen and (min-width: 800px){.rutimezone[_ngcontent-sc122]{font-weight:bold;text-align:right}}@media only screen and (max-width: 800px){.rutimezone[_ngcontent-sc122]{font-weight:bold;text-align:left}}</style></head><body><meta\xa0http-equiv="x-ua-compatible"\xa0content="ie=edge">\n  <meta charset="utf-8">\n  <title>S&amp;P Global Ratings</title>\n  <base href="/ratings">\n  <meta name="viewport" content="width=device-width, initial-scale=1">\n  <meta http-equiv="X-UA-Compatible" content="IE=edge">\n\n  <link rel="icon" type="image/x-icon" href="ratings/favicon.i

In [ ]:
# Enrichment
#1. Add to each index the current value
# Get from https://www.finanzen.net/anleihen/a0e6fu-suedzucker-international-finance-bv-anleihe
#2. Add to each index the Rating
# Get from https://www.finanzen.net/anleihen/a0e6fu-suedzucker-international-finance-bv-anleihe
#3. Add to each index the sales value of the last day

# Helper Function

In [ ]:
from datetime import datetime

datetime_str = '12.12.2024'

datetime_object = datetime.strptime(datetime_str, '%d.%m.%Y')

print(type(datetime_object))
print(datetime_object) 